# <b>CONSUMO DE ENERGÍA ELÉCTRICA

American Electric Power (AEP), es una importante empresa de servicios públicos de propiedad de inversores en los Estados Unidos, que suministra electricidad a más de cinco millones de clientes en 11 estados.

AEP se encuentra entre los generadores de electricidad más grandes del país y posee casi 38,000 megavatios de capacidad de generación en los EE. UU.

In [1]:
# Librerias necesarias
# ==============================================================
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

In [15]:
# Carga de los datos 
AEP = pd.read_csv('DATA/COMED_hourly.csv')

### <b>Análisis Exploratorio

In [16]:
AEP.head()

,Datetime,COMED_MW
0,2011-12-31 01:00:00,9970.0
1,2011-12-31 02:00:00,9428.0
2,2011-12-31 03:00:00,9059.0
3,2011-12-31 04:00:00,8817.0
4,2011-12-31 05:00:00,8743.0


In [17]:
AEP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66497 entries, 0 to 66496
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Datetime  66497 non-null  object 
 1   COMED_MW  66497 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.0+ MB


In [18]:
# Valores nulos
AEP.isnull().sum()

Datetime    0
COMED_MW    0
dtype: int64

In [19]:
# Cambio de nombre a columna
AEP.columns = ['datetime','mw']
# Conviertiendo a tipo datetime
AEP['datetime'] = pd.to_datetime(AEP['datetime'])

In [20]:
AEP

,datetime,mw
0,2011-12-31 01:00:00,9970.0
1,2011-12-31 02:00:00,9428.0
2,2011-12-31 03:00:00,9059.0
3,2011-12-31 04:00:00,8817.0
4,2011-12-31 05:00:00,8743.0
...,...,...
66492,2018-01-01 20:00:00,13858.0
66493,2018-01-01 21:00:00,13758.0
66494,2018-01-01 22:00:00,13627.0
66495,2018-01-01 23:00:00,13336.0


Contamos con 121,273 registros, que comprenden el consumo eléctrico por hora en megavatios. Estos abarcan un período desde el día 1 de octubre de 2004 hasta el día 3 de Agosto de 2018. Sin registros nulos.

In [21]:
AEP.sort_values('datetime', inplace=True)
# Columna de años
AEP['anio'] = AEP['datetime'].dt.year
# Columna de cuartos
AEP['quarter'] = AEP['datetime'].dt.quarter
# Columna de meses
AEP['mes'] = AEP['datetime'].dt.month
# Columna de horas
AEP['hora'] = AEP['datetime'].dt.hour
# Estableciendo la variable 'Datetime' como indice
AEP = AEP.set_index('datetime')
#AEP = AEP.asfreq('1H')
AEP

,mw,anio,quarter,mes,hora
datetime,,,,,
2011-01-01 01:00:00,9631.0,2011,1,1,1
2011-01-01 02:00:00,9273.0,2011,1,1,2
2011-01-01 03:00:00,9011.0,2011,1,1,3
2011-01-01 04:00:00,8741.0,2011,1,1,4
2011-01-01 05:00:00,8694.0,2011,1,1,5
...,...,...,...,...,...
2018-08-02 20:00:00,16437.0,2018,3,8,20
2018-08-02 21:00:00,15590.0,2018,3,8,21
2018-08-02 22:00:00,15086.0,2018,3,8,22


In [22]:
# Verificar que un índice temporal está completo
# ==============================================================================
(len(AEP.index) == pd.date_range(
                    start = AEP.index.min(),
                    end   = AEP.index.max(),
                    freq  = 'H')
).all()

False

In [23]:
pd.date_range(
                    start = AEP.index.min(),
                    end   = AEP.index.max(),
                    freq  = 'H')

66497

In [ ]:
meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']

## Distribución de demanda por mes

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(24,12))
plt.title("DEMANDA POR MES", fontsize=24)
sns.lineplot(x='mes', y='MW', hue='anio', data=AEP, ci=None, ax=axes[0])
plt.xticks(np.arange(1,13), meses)

sns.boxplot(x='mes', y='MW', data=AEP, ax=axes[1])
demanda_mes = AEP.groupby('mes')['MW'].median().reset_index()
sns.lineplot(x=demanda_mes.mes, y=demanda_mes.MW, ax=axes[1])
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x=demanda_mes.mes, y=demanda_mes.MW)



Podemos observar que existe 

## Tendencia de consumo por cuatrimestre

In [ ]:
fig, ax = plt.subplots(figsize=(24,6))
plt.title("CONSUMO ELÉCTRICO PROMEDIO POR CUARTO", fontsize=18)
sns.lineplot(x='quarter', y='MW', hue='anio', data=AEP, marker='o')
plt.xticks(np.arange(1,5), ['Q1','Q2','Q3','Q4'])

fig, ax = plt.subplots(figsize=(24,6))
sns.boxplot(x='quarter', y='MW', data=AEP)
plt.xticks(np.arange(1,5), ['Q1','Q2','Q3','Q4'])
plt.show()

## Tendencias de consumo por horas

In [ ]:
plt.figure(figsize=(24, 6))
plt.title("CONSUMO ELÉCTRICO PROMEDIO POR HORA", fontsize=24)
sns.lineplot(x='hora', y='MW', hue='quarter', data=AEP, marker='o')
plt.xticks(range(24))
plt.show()

In [ ]:
plt.figure(figsize=(24, 6))
plt.title("CONSUMO ELÉCTRICO PROMEDIO POR HORA/CUARTOS", fontsize=24)
sns.lineplot(x='hora', y='MW', hue='anio', data=AEP, marker="o", palette='Dark2')
plt.xticks(range(24))
plt.grid()
plt.show()